In [5]:
import pandas as pd
import json
import datetime
import requests
import numpy as np
import math
import base64
import from fuzzywuzzy import process

# Table of Contents
## 1 TicketMaster Face Value Data
## 2 TicketMaster Resale Data
## 3 SeatGeek Data
## 4 StubHub Data
## 5 Join DataFrames

## 1 TicketMaster Data
### List of cities and Ticketmaster codes
City_List = 
    'Denver' : 264,
    'San Francisco Bay': 382,
    'Portland': 362,
    'Los Angeles': 27,
    'Las Vegas' : 14,
    'Phoenix': 36, 
    'Seattle': 42,
    'Austin' : 40, 
    'Houston': 22,
    'Dallas' : 5,
    'Chicago': 3,
    'Nashville': 31,
    'Atlanta': 10,
    'Boston' : 11,
    'New York': 35,
    'Washington DC': 47,
    'Miami' : 15
### Ticketmaster API info    
base_url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&apikey={apikey}'
api_key = 'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'
dmaId = [382,362,264]
marketId =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]



### 1.1 Define Functions

Define functions for pulling event data from Ticketmaster API and formatting dataframe

In [7]:
#DMA and Market numbers
dma_list = [382,362,264]
mkt_list =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]

# Function to get the number of pages of ticketmaster data
def get_number_of_TM_pages(dma=None,market=None,source='ticketmaster,frontgate'):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': source,
               'classificationName': 'music',
               'size': '200',
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    return json_obj['page']['totalPages']

#Get TicketMaster data, return a dataframe
def getTicketMasterData(dma=None,market=None,page=None, source='ticketmaster,frontgate'):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': source,
               'classificationName' : 'music',
               'sort': 'date,name,asc',
               'size': '200',
               'page': page,
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_response = json.loads(r.text)
    event_info = []
    for event in json_response.get('_embedded',{}).get('events',{}):
         event_info.append({
            'TM_id': event.get('id',{}),
            'TM_name' : event.get('name',{}),
            'TM_artist': list(attraction.get('name') for attraction in event['_embedded'].get('attractions',{})),
            'TM_venue' : list(venue.get('name') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_city' : list(venue.get('city',{}).get('name') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_state' : list(venue.get('state',{}).get('stateCode') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_description' : event.get('description',{}),
            'TM_more_info' : event.get('additionalInfo',{}),
            'TM_start_date' : event.get('dates',{}).get('start',{}).get('dateTime',{}),
            'TM_timezone' : event.get('dates',{}).get('timezone'),
            'TM_span_multiple_days' : event.get('dates',{}).get('spanMultipleDays'),
            'TM_presale_date_start' : list(presale.get('startDateTime',{}) for presale in event.get('sales').get('presales',{})),
            'TM_presale_date_end' : list(presale.get('endDateTime',{}) for presale in event.get('sales').get('presales',{})),
            'TM_sale_date_start' : event.get('sales',{}).get('public',{}).get('startDateTime'),
            'TM_FV_prices': event.get('priceRanges'),
            'TM_promoter': event.get('promoter',{}).get('name'),
            'TM_genre' : event.get('classifications'),
            'TM_place' : event.get('place')
         })
    tmDF = pd.DataFrame(event_info)
    return tmDF

#Convert timedate information from UTC to local time
def convert_times(df,times_list,tz_col):
    #Loop through each timezone
    df_list = []
    for tz in df[tz_col].unique():
        #Filter rows by timezone
        mask = (df[tz_col] == tz)
        df_local = df.loc[mask]
        #Loop through each datetime row
        for col in times_list:
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
            df_local[col] = pd.to_datetime(df_local[col],errors='coerce').dt.tz_localize('UTC').dt.tz_convert(tz)
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
        df_list.append(df_local)
    df = pd.concat(df_list, axis=0)
    return df

#Explode out columns with nested information
def explode(df,col,index_col):
    df1 = df[col].apply(pd.Series)
    df1 = df1[0].apply(pd.Series)
    df1.index = df[index_col]
    df = df.merge(df1, how='left', left_on=index_col, right_index=True)
    df = df.drop(col,axis=1)
    return df

### 1.2 Query Ticketmaster face value data
### Pull the data and create the data frame
For each city/market, get the number of pages of data, pull all data, and then concatanate the dataframes

In [ ]:
fv_df_list = []

#Create dictionaries of dma/mkt numbers, and number of pages of data
dma_dict = dict()
mkt_dict = dict()
#Get number of pages for face value dma items
for dma in dma_list:
    dma_dict[dma] = get_number_of_TM_pages(dma=dma)
    
#Get DMA data
for dma in dma_dict.keys():
    for page in range(1,dma_dict[dma]+1):
        fv_df_list.append(getTicketMasterData(dma=dma,page=page))
    
#Get number of pages for face value market items
for mkt in mkt_list:
    mkt_dict[mkt] = get_number_of_TM_pages(market=mkt)
    
#Get market data
for mkt in mkt_dict.keys():
    for page in range(1,mkt_dict[mkt]+1):
        fv_df_list.append(getTicketMasterData(market=mkt,page=page))
        
#Combine all dataframes for full data on events and face_value prices
fv_df = pd.concat(fv_df_list,axis=0)

fv_df.head()

### 1.3 Convert datetime columns to datetime objects in  proper timezones
Currently the four columns with datetime information are a mess. Some are of type dict, some are nested in lists, and all are in the UTC timezone. We can look and see that in timedate columns with lists of dates, the timedates are identitical, so we can unnest datetimes by simply taking the first element of the list.

The 3 things we need to accomplish:
    1. Convert all datetime columns to dtype string, and unnest 'TM_presale_date_end' and 'TM_presale_date_start' columns
    2. Convert all columns to datetime series localized to UTC (Done in the convert_times function)
    3. Filter rows by timezone, and loop over each datetime column to convert objects to their proper timezone (Done in the convert_times function)

In [ ]:
# 1. Convert all datetime columns to dtype string, and unnest objects
fv_df['TM_presale_date_end'] = fv_df['TM_presale_date_end'].str[0]
fv_df['TM_presale_date_start'] = fv_df['TM_presale_date_start'].str[0]
fv_df['TM_start_date'] = fv_df['TM_start_date'].astype(str)
fv_df['TM_sale_date_start'] = fv_df['TM_sale_date_start'].astype(str)


In [ ]:
# 2. Convert all columns to datetime series localized to UTC
# 3. Filter rows by timezone and create a new dataframe per timezone, loop over each datetime column to convert objects to their proper timezone 
col_list = ['TM_presale_date_end','TM_presale_date_start','TM_start_date','TM_sale_date_start']
fv_df = convert_times(df=fv_df,times_list=col_list,tz_col='TM_timezone')

### 1.4 Explode nested rows
Unnest information in prices and genre column

In [ ]:
fv_df.head()

In [ ]:
#Explode price column
fv_df = explode(df=fv_df,col='TM_FV_prices',index_col='TM_id')
fv_df.head()

In [ ]:
fv_df'TM_genre'].keys()

In [ ]:
#Unnest genre info and clean up data in exploded columns
#fv_df = explode(fv_df,'TM_genre','TM_id')
# for col in ['genre','subType']:
#     fv_df[col] = fv_df[col].map(lambda x: x.get('name',{}))
# fv_df.head()

## 2 Query TicketMaster resale data and merge with face value data
Next we will query TicketMaster's resale ticket data and then merge it with the face value dataframe

In [ ]:
#Query resale data from the Ticketmaster API

rv_df_list = []

#Create dictionaries of dma/mkt numbers, and number of pages of data
dma_rv_dict = dict()
mkt_rv_dict = dict()
#Get number of pages for resale value dma items
for dma in dma_list:
    dma_rv_dict[dma] = get_number_of_TM_pages(dma=dma,source='tmr')
    
#Get DMA data
for dma in dma_rv_dict.keys():
    for page in range(1,dma_dict[dma]+1):
        fv_df_list.append(getTicketMasterData(dma=dma,page=page,source='tmr'))
    
#Get number of pages for resale value market items
for mkt in mkt_list:
    mkt_rv_dict[mkt] = get_number_of_TM_pages(market=mkt,source='tmr')
    
#Get market data
for mkt in mkt_rv_dict.keys():
    for page in range(1,mkt_dict[mkt]+1):
        fv_df_list.append(getTicketMasterData(market=mkt,page=page,source='tmr'))
        
#Combine all dataframes for full data on events and resale value prices
rv_df = pd.concat(fv_df_list,axis=0)
rv_df.head()

### 2.1 Extract price information and merge it with face value data in a new dataframe

In [ ]:
# Explode out resale price information
rv_df = explode(rv_df,'TM_FV_prices','TM_id')
rv_df.rename(columns={'min': 'tmr_min', 'max': 'tmr_max'}, inplace=True)
# Create a new dataframe combining face value and resale value information, joined on ticketmaster event ID
joined_df = fv_df.join(rv_df[['tmr_min','tmr_max']],on='TM_id',how='outer')
joined_df.head()

## 3 Download Resale Data from SeatGeek API
1. Connect to SeatGeek API and download data
2. Merge data with TicketMaster DataFrame

### 3.1 Define functions

In [ ]:
#Connect to SeatGeek API and determine number of pages of information
def get_SeatGeek_Pages():
    url = 'https://api.seatgeek.com/2/events?format=json'
    payload = {'per_page' : 1000,
               'taxonomies.name':'concert',
               'client_id': 'OTU5MDE5MXwxNTEwMzcxNjgyLjIx',
              }
    r = requests.get(url, params=payload,verify=True)
    json_obj = json.loads(r.text)
    #Return the total number of JSON items divided by the number of page to get page count
    return math.ceil(json_obj['meta']['total']/json_obj['meta']['per_page'])

#Connect to SeatGeek API and download JSON data, format it into pandas dataframe
def get_SeatGeek_data(page=1):
    url = 'https://api.seatgeek.com/2/events?format=json'
    payload = {'per_page' : 1000,
               'page' : page,
               'taxonomies.name':'concert',
               'client_id': 'OTU5MDE5MXwxNTEwMzcxNjgyLjIx',
              }
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    info_list = []
    for event in json_obj['events']:
        info_list.append(
         {'SG_listing_count' : event.get('stats',{}).get('listing_count',{}),
         'SG_average_price' : event.get('stats',{}).get('average_price',{}),
         'SG_lowest_price' : event.get('stats',{}).get('lowest_price',{}),
         'SG_highest_price' : event.get('stats',{}).get('highest_price',{}),
         'SG_title' : event.get('title',{}),
         'SG_datetime_local' : event.get('datetime_local',{}),
         'SG_artists' : [performer.get('name',{}) for performer in event.get('performers',{})],
         'SG_artists_score' : [performer.get('score',{}) for performer in event.get('performers',{})],
         'SG_artists_type' : [performer.get('type',{}) for performer in event.get('performers',{})],
         'SG_venue' : event.get('venue',{}).get('name',{}),
         'SG_venue_city' : event.get('venue',{}).get('city',{}),
         'SG_venue_state' : event.get('venue',{}).get('state',{}),
         'SG_venue_score' : event.get('venue',{}).get('score',{})})
    sgdf = pd.DataFrame(info_list)
    return sgdf

### 3.2 Download SeatGeek data into DataFrame

In [ ]:
sgdf_list = []
#Find the total number of pages in 
total_pages = get_SeatGeek_Pages()
#Loop through the number pages of data and combine data into single dataframe
for pageNum in range(1,total_pages+1):
    sgdf_list.append(get_SeatGeek_data(page=pageNum))
sgdf = pd.concat(sgdf_list,axis=0)
sgdf.head()

## 4 StubHub Data
Connect to StubHub API and download data

### 4.1 Define Functions

In [3]:
access_token = '57476f0a-f69e-334d-ba1b-02d394883b2a'
user_GUID = '84175DB4D85A6777E04400144FB7AE36'

#Get number of pages of data
def get_SH_pages():
    url = 'https://api.stubhub.com/search/catalog/events/v3/'
    payload = {'minAvailableTickets':1, 
              'categoryName':'Concert',
              'country' : 'US'}
    headers = {'Authorization': 'Bearer ' + access_token} # Insert StubHub API Key here
    r = requests.get(url, params=payload, headers=headers, verify=True)
    json_response = json.loads(r.text)
    return math.ceil(json_response.get('numFound',{})/500)

#Connect to StubHub API and put data into DataFrame
def get_SH_data(start):
    url = 'https://api.stubhub.com/search/catalog/events/v3/'
    payload = {'minAvailableTickets':1, 
              'categoryName':'Concert',
               'rows' : 500,
               'start' : start,
               'country' : 'US'}
    headers = {'Authorization': 'Bearer ' + access_token} # Insert StubHub API Key here
    r = requests.get(url, params=payload, headers=headers, verify=True)
    json_response = json.loads(r.text)
    event_info = []
    for event in json_response.get('events',{}):
            event_info.append({
                    'SH_artist':event.get('ancestors',{}).get('performers',{}),
                    'SH_date':str(event.get('eventDateLocal',{})),
                    'SH_event_id':str(event.get('id',{})),
                    'SH_min_price':str(event.get('ticketInfo',{}).get('minPrice',{})),
                    'SH_max_price':str(event.get('ticketInfo',{}).get('maxPrice')),
                    'SH_total_postings':str(event.get('ticketInfo',{}).get('totalPostings')),
                    'SH_total_tickets':str(event.get('ticketInfo',{}).get('totalTickets')),
                    'SH_venue':str(event['venue'].get('name')),
                    'SH_city':str(event['venue'].get('city')),
                    'SH_state':str(event['venue'].get('state'))
                })
    TicketData = pd.DataFrame(event_info)
    return TicketData

### 4.2 Gather data and concat into a dataframe

In [5]:
pages = get_SH_pages()
SH_df_list = []
for page in range(pages):
     start_place = page * 500
     SH_df_list.append(get_SH_data(start=start_place))
SH_df = pd.concat(SH_df_list,axis=0)

SH_df.head()

,SH_artist,SH_city,SH_date,SH_event_id,SH_max_price,SH_min_price,SH_state,SH_total_postings,SH_total_tickets,SH_venue
0,"[{'id': 5984, 'name': 'Bruce Springsteen', 'ur...",New York,2017-12-29T20:00:00-0500,103170465,6668.5,2030.5,NY,32,59,Walter Kerr Theatre
1,"[{'id': 5984, 'name': 'Bruce Springsteen', 'ur...",New York,2018-01-09T20:00:00-0500,103170471,9602.5,3002.5,NY,21,37,Walter Kerr Theatre
2,"[{'id': 492049, 'name': 'Black Veil Brides', '...",Seattle,2018-02-24T20:00:00-0800,103228116,1362.8,55.3,WA,16,62,Showbox Sodo
3,"[{'id': 196736, 'name': 'Joe Bonamassa', 'url'...",Chattanooga,2017-12-04T20:00:00-0500,9864305,919.3,121.3,TN,15,46,Soldiers and Sailors Memorial Auditorium
4,"[{'id': 712224, 'name': 'Walk Off The Earth', ...",Las Vegas,2018-03-03T19:00:00-0800,103271420,91.3,60.08,NV,6,41,House of Blues Las Vegas


### 4.3 Convert to datetimes, Explode out artist names

In [6]:
#From string to datetime
SH_df['SH_date'] = pd.to_datetime(SH_df['SH_date'])

#Explode out artists column
SH_df = explode(SH_df,'SH_artist','SH_event_id')

28

## 5 Join dataframes on venue name fuzzy match and datetime
### 5.1 Define Functions

In [ ]:
#Create a function to find fuzzy matches from one DF to another

